In [82]:
import pickle
import numpy as np
import pandas as pd
from pprint import pprint
from time import time
import logging
import sys
import tensorflow as tf
from os import path
from os import listdir
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow import feature_column
#import tensorflow_hub as hub
from tensorflow.keras.callbacks import TensorBoard
from azureml.core import Dataset, Workspace
import joblib
#import tensorflow_addons as tfa

state_gov_nm = 'MA'
year_to_process = '2020'
data_file_location = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/beefy/code/QuigleyData'

vector_size = 32
cluster_size = 50

#state_gov_nm = sys.argv[1]
#year_to_process = sys.argv[2]
#data_file_location = sys.argv[3]


In [83]:
#ws = Workspace.get(name="ml-quigley"
#                   , subscription_id='8d8bcbfe-86b4-42bb-a470-a8cf97c98d69'
#                   , resource_group='project-quigley')

#local_ds_path = tempfile.mkdtemp()

#define the azureml dataset
#leg_meta_ds = Dataset.get_by_name(ws,'leg_meta_ds')

In [84]:
#Create features for text
#module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

#Model load through local path:
#module_path = data_file_location + '/universal-sentence-encoder_4'
#%time embed = hub.load(module_path)

In [85]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
#labels are derived from the "passed" column
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('passed')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [86]:
_dataset = path.join(data_file_location, state_gov_nm + str(vector_size) + str(cluster_size) + '_dataset.df')
with open(_dataset, 'rb') as f:
    leg_data = pickle.load(f)

train_set = leg_data[leg_data['year']<int(year_to_process)]
test_set = leg_data[leg_data['year']==int(year_to_process)]

test = test_set

#create a balanced train/test split

data_pass = leg_data[leg_data['passed']==1]
data_fail = leg_data[leg_data['passed']==0]

if len(data_pass) > len(data_fail):
    data_pass  = data_pass.iloc[:len(data_fail)]
else:
    data_fail = data_fail.iloc[:len(data_pass)]

print(len(data_pass))
print(len(data_fail))

#combine the balanced sets and shuiffle
balanced_train = pd.concat([data_pass, data_fail])
balanced_train = balanced_train.sample(frac=1)
balanced_train = balanced_train.sample(frac=1)

#pull out a validation set
train, val = train_test_split(balanced_train, test_size=.25)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


41265
41265
61897 train examples
20633 validation examples
41964 test examples


In [87]:
#only for previous years
#test = test.sample(frac=1)
#test = test.sample(frac=1)
#print(test.head())
#test = test.drop_duplicates(subset='bill_id')

In [88]:
idx = test.groupby(['bill_id'])['time_t'].transform(max) == test['time_t']
test = test[idx]
print(len(test))

7717


In [89]:
len(train['time_t'].unique())

107

In [90]:
feature_columns = []

numericfeaturelist = [
                        'sponsor_vec', #change to -1 to 1
                        'partisan_vector'
                      ]

for dv in range(1, vector_size+1):
    numericfeaturelist.append('dv' + str(dv))

sponsors_cols = [col for col in leg_data.columns if 'sponsors_' in col]

numericfeaturelist.extend(sponsors_cols)

#numeric cols
for header in numericfeaturelist:
    feature_columns.append(feature_column.numeric_column(header))

# indicator cols
primary_party = feature_column.categorical_column_with_vocabulary_list('primary_party', leg_data['primary_party'].unique())
primary_party_onehot = feature_column.indicator_column(primary_party)
feature_columns.append(primary_party_onehot)

time_t = feature_column.categorical_column_with_vocabulary_list('time_t', leg_data['time_t'].unique())
time_t_onehot = feature_column.indicator_column(time_t)
feature_columns.append(time_t_onehot)

primary_sponsor = feature_column.categorical_column_with_vocabulary_list('primary_name', leg_data['primary_name'].unique())
primary_sponsor_onehot = feature_column.indicator_column(primary_sponsor)
feature_columns.append(primary_sponsor_onehot)

bill_category = feature_column.categorical_column_with_vocabulary_list('category', leg_data['category'].unique())
bill_category_onehot = feature_column.indicator_column(bill_category)
feature_columns.append(bill_category_onehot)

# embedding cols
event_list = feature_column.categorical_column_with_vocabulary_list('cmltv_event', leg_data['cmltv_event'].unique())
event_list_onehot = feature_column.indicator_column(event_list)
event_list_embed = feature_column.embedding_column(event_list, dimension=32)

#feature_columns.append(event_list_embed)
feature_columns.append(event_list_onehot)
#primary_embedding = feature_column.embedding_column(primary_sponsor, dimension=8)

print(len(feature_columns))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)


440


In [91]:

batch_size = 512
train_ds = df_to_dataset(train, shuffle = True, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=True, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [92]:
model = tf.keras.Sequential([
      feature_layer,
      layers.Dense(1000, activation='relu'),
      layers.Dropout(.5),
      layers.Dense(250, activation='relu'),
      layers.Dropout(.25),
      layers.Dense(1000, activation='relu'),
      layers.Dropout(.5),
      layers.Dense(1, activation='sigmoid')
    ])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'
                       , tf.keras.metrics.Precision()
                       , tf.keras.metrics.Recall()
                      # , tfa.metrics.F1Score()
                      ])

model.fit(train_ds,
          #validation_data = test_ds,
          validation_data=val_ds,
          epochs=15#, callbacks = [tensorboard]
         )


Train for 121 steps, validate for 41 steps
Epoch 1/15
121/121 [==============================] - 79s 655ms/step - loss: 0.5477 - accuracy: 0.8974 - precision_6: 0.9471 - recall_6: 0.8419 - val_loss: 0.5245 - val_accuracy: 0.9535 - val_precision_6: 0.9699 - val_recall_6: 0.9360
Epoch 2/15
121/121 [==============================] - 50s 411ms/step - loss: 0.5248 - accuracy: 0.9515 - precision_6: 0.9703 - recall_6: 0.9314 - val_loss: 0.5237 - val_accuracy: 0.9515 - val_precision_6: 0.9814 - val_recall_6: 0.9204
Epoch 3/15
121/121 [==============================] - 61s 507ms/step - loss: 0.5206 - accuracy: 0.9606 - precision_6: 0.9767 - recall_6: 0.9438 - val_loss: 0.5214 - val_accuracy: 0.9573 - val_precision_6: 0.9864 - val_recall_6: 0.9274
Epoch 4/15
121/121 [==============================] - 50s 411ms/step - loss: 0.5189 - accuracy: 0.9648 - precision_6: 0.9793 - recall_6: 0.9498 - val_loss: 0.5185 - val_accuracy: 0.9667 - val_precision_6: 0.9806 - val_recall_6: 0.9521
Epoch 5/15
121/12

In [93]:
#export the model
try:
    _model = path.join(data_file_location, 'model/' + state_gov_nm + '/'+ state_gov_nm + 'overall.tf')
    model.save(_model)
except:
    print("Could not save model.")

In [94]:
_predictions = path.join(data_file_location, 'predictions/' + state_gov_nm + year_to_process + 'predictions.csv')
predictions = model.predict(test_ds)
test['pred'] = predictions
test.to_csv(_predictions)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
